In [1]:
import glob
import warnings
import tarfile
import cv2

import keras
import keras.backend as K

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, LeakyReLU
from keras.layers import BatchNormalization, Activation, Conv2D
from keras.layers import GlobalAveragePooling2D, Lambda
from keras.optimizers import Adam, RMSprop

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical, Sequence
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from keras.models import load_model, model_from_json

import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
from PIL import Image
from cv2 import resize

import tensorflow as tfw
from keras.applications import ResNet50
from keras import regularizers
import requests
import threading
import random
import time
import urllib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import multi_gpu_model

from collections import Counter

%matplotlib inline

print('Keras version:', keras.__version__)
# print(os.listdir('SageMaker'))

warnings.simplefilter('default')
!pwd

Using TensorFlow backend.


Keras version: 2.2.4
/home/ec2-user/SageMaker


In [ ]:
train_path = './data/landmarks/train/train/'
test_path = './data/landmarks/test/test/'
train_images = glob.glob(train_path+'*.jpg')
test_images = glob.glob(test_path+'*.jpg')
print(len(train_images))
print(len(test_images))
sample_submission = pd.read_csv('./data/landmarks/recognition_sample_submission.csv')
sample_submission.shape

In [ ]:
train_image_ids = [image_file.replace(
    '.jpg', '').replace(train_path, '') for image_file in train_images]

train_df = pd.DataFrame(index=list(range(0,len(train_image_ids))))
train_df['filename'] = pd.Series(train_images, index=list(range(0,len(train_image_ids))))
train_df['ids'] = train_image_ids
test_image_ids = [image_file.replace(
    '.jpg', '').replace(test_path, '') for image_file in test_images]
test_df = pd.DataFrame(index=list(range(0,len(test_image_ids))))
test_df['filename'] = pd.Series(test_images, index=list(range(0,len(test_image_ids))))
test_df['ids'] = test_image_ids

In [ ]:
train = pd.read_csv("./data/train.csv",index_col='id')
print(train.head())
print(train.shape)
print("Number of classes {}".format(len(train.landmark_id.unique())))

NUM_THRESHOLD = 420

counts = dict(Counter(train['landmark_id']))
landmarks_dict = {x:[] for x in train.landmark_id.unique() if counts[x] >= NUM_THRESHOLD}
NUM_CLASSES = len(landmarks_dict)
print("Total number of valid classes: {}".format(NUM_CLASSES))

i = 0
landmark_to_idx = {}
idx_to_landmark = []
for k in landmarks_dict:
    landmark_to_idx[k] = i
    idx_to_landmark.append(k)
    i += 1
    
train['filename'] = pd.Series(train_images, index=train_image_ids)

all_urls = train['url'].tolist()
all_landmarks = train['landmark_id'].tolist()
valid_urls_dict = {x[0].split("/")[-1]:landmark_to_idx[x[1]] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict}
valid_urls_list = [x[0] for x in zip(all_urls, all_landmarks) if x[1] in landmarks_dict]

NUM_EXAMPLES = len(valid_urls_list)
print("Total number of valid examples: {}".format(NUM_EXAMPLES)
     )

In [ ]:
test_info_full = pd.read_csv('./data/test.csv', index_col='id')
test_info_full.head()

test_info = test_info_full.loc[test_image_ids]
test_info['filename'] = pd.Series(test_images, index=test_image_ids)

test_info.head()